In [1]:
print("all ok")

all ok


### Sequential flow

In [3]:
from dotenv import load_dotenv
import os
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.teams import GraphFlow,DiGraphBuilder
from autogen_ext.models.openai import OpenAIChatCompletionClient

load_dotenv()
os.environ["OPENAI_API_KEY"]=os.getenv("OPENAI_API_KEY")

In [4]:
model_client=OpenAIChatCompletionClient(model='gpt-4.1-nano')

In [5]:
writer_agent=AssistantAgent(
    name="writer",
    model_client=model_client,
    system_message="You are a helpful writer , write something about the query within 100 words."
)

reviewer_agent=AssistantAgent(
    name="reviewer",
    model_client=model_client,
    system_message="You are a helpful reviewer , review the previous run output draft and suggest for improvement."
)

In [6]:
builder=DiGraphBuilder()
builder.add_node(writer_agent).add_node(reviewer_agent)
builder.add_edge(writer_agent,reviewer_agent)

In [7]:
graph=builder.build()

In [8]:
graph

DiGraph(nodes={'writer': DiGraphNode(name='writer', edges=[DiGraphEdge(target='reviewer', condition=None, condition_function=None, activation_group='reviewer', activation_condition='all')], activation='all'), 'reviewer': DiGraphNode(name='reviewer', edges=[], activation='all')}, default_start_node=None)

In [9]:
graphflow=GraphFlow([writer_agent,reviewer_agent],graph)

In [10]:
graphflow

In [11]:
stream=graphflow.run_stream(task="Write a short paragraph about climate change.")

In [12]:
async for event in stream:
    print(event)

id='fc37703a-f5c2-4a4e-a98d-98c5b3c6cdb0' source='user' models_usage=None metadata={} created_at=datetime.datetime(2025, 7, 25, 5, 47, 9, 451667, tzinfo=datetime.timezone.utc) content='Write a short paragraph about climate change.' type='TextMessage'
id='6287fbb0-da0f-4a17-b83f-d46d892864f7' source='writer' models_usage=RequestUsage(prompt_tokens=36, completion_tokens=112) metadata={} created_at=datetime.datetime(2025, 7, 25, 5, 47, 12, 582211, tzinfo=datetime.timezone.utc) content='Climate change refers to long-term shifts in weather patterns and global temperatures, primarily caused by human activities such as burning fossil fuels, deforestation, and industrial processes. These changes lead to more frequent and severe weather events, rising sea levels, melting glaciers, and disrupted ecosystems. Climate change poses significant threats to biodiversity, agriculture, and human health worldwide. Addressing it requires urgent collective action through reducing greenhouse gas emissions, a

### Parallel flow with join

In [13]:
writer_agent=AssistantAgent(
    name='writer',
    model_client=model_client,
    system_message="Draft a short paragraph with 100 words on the given subject."
)

editor_1_agent=AssistantAgent(
    name="editor1",
    model_client=model_client,
    system_message="Edit the paragraph for grammar."
)
editor_2_agent=AssistantAgent(
    name="editor2",
    model_client=model_client,
    system_message="Edit the paragraph for style."
)

final_reviewer_agent=AssistantAgent(
    name='final_reviewer',
    model_client=model_client,
    system_message="Consolidate the grammar and style edits into a final version."
)


In [15]:
builder=DiGraphBuilder()
builder.add_node(writer_agent).add_node(editor_1_agent).add_node(editor_2_agent).add_node(final_reviewer_agent)
builder.add_edge(writer_agent,editor_1_agent)
builder.add_edge(writer_agent,editor_2_agent)

builder.add_edge(editor_1_agent,final_reviewer_agent)
builder.add_edge(editor_2_agent,final_reviewer_agent)

graph=builder.build()

graphflow=GraphFlow(participants=builder.get_participants(),
                    graph=graph)

In [16]:
from autogen_agentchat.ui import Console

In [17]:
await Console(graphflow.run_stream(task="write a paragraph on climet change"))

---------- TextMessage (user) ----------
write a paragraph on climet change
---------- TextMessage (writer) ----------
Climate change refers to long-term shifts in weather patterns caused primarily by human activities, such as burning fossil fuels, deforestation, and industrial processes. These actions increase greenhouse gases like carbon dioxide and methane in the atmosphere, trapping heat and leading to global warming. The effects of climate change include rising sea levels, more intense and frequent natural disasters, melting glaciers, and disrupted ecosystems. It poses significant threats to biodiversity, food security, and human health worldwide. Addressing climate change requires urgent global cooperation to reduce emissions, adopt sustainable practices, and transition to renewable energy sources for a healthier planet and future generations.
---------- TextMessage (editor1) ----------
Climate change refers to long-term shifts in weather patterns caused primarily by human activi

TaskResult(messages=[TextMessage(id='809cb138-7045-41de-9703-1d77d035ecba', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 7, 25, 5, 59, 9, 457977, tzinfo=datetime.timezone.utc), content='write a paragraph on climet change', type='TextMessage'), TextMessage(id='4580adbb-bc77-48e4-ab54-d0afaba6d90f', source='writer', models_usage=RequestUsage(prompt_tokens=32, completion_tokens=120), metadata={}, created_at=datetime.datetime(2025, 7, 25, 5, 59, 12, 56234, tzinfo=datetime.timezone.utc), content='Climate change refers to long-term shifts in weather patterns caused primarily by human activities, such as burning fossil fuels, deforestation, and industrial processes. These actions increase greenhouse gases like carbon dioxide and methane in the atmosphere, trapping heat and leading to global warming. The effects of climate change include rising sea levels, more intense and frequent natural disasters, melting glaciers, and disrupted ecosystems. It poses sign

### Parallel Fan out

In [19]:
agent_A=AssistantAgent(
    name='A',
    model_client=model_client,
    system_message="Write a story on the given subject."
)

agent_B=AssistantAgent(
    name='B',
    model_client=model_client,
    system_message="Translate the given text in Chinese language."
)

agent_C=AssistantAgent(
    name="C",
    model_client=model_client,
    system_message="Translate the text into Japanese language."

)

agent_D=AssistantAgent(
    name='D',
    model_client=model_client,
    system_message="Translate the text in Hindi language."
)

agent_E=AssistantAgent(
    name="E",
    model_client=model_client,
    system_message="Translate the text into Bengali language and summarize it with in 100 words."
)

In [20]:
from autogen_agentchat.conditions import MaxMessageTermination

In [26]:
builder=DiGraphBuilder()
builder.add_node(agent_A).add_node(agent_B).add_node(agent_C).add_node(agent_D).add_node(agent_E)

builder.add_edge(agent_A,agent_B)
builder.add_edge(agent_A,agent_C)
builder.add_edge(agent_A,agent_D)

builder.add_edge(agent_D,agent_E)

graph=builder.build()

graphflow=GraphFlow(participants=builder.get_participants(),
                    graph=graph,
                    termination_condition=MaxMessageTermination(10))

In [27]:
task="Write a short story about a cat."

In [28]:
stream=graphflow.run_stream(task=task)

In [29]:
await Console(stream=stream)

---------- TextMessage (user) ----------
Write a short story about a cat.
---------- TextMessage (A) ----------
Once upon a time, in a cozy little village, there lived a curious cat named Whiskers. With sleek gray fur and bright green eyes that sparkled like emeralds, Whiskers loved to explore every nook and cranny of his neighborhood.

Every morning, he prowled through gardens, climbed trees, and chased fluttering butterflies. But what Whiskers loved most was visiting the old wooden library at the edge of the village. It was a place filled with stories and secrets, and Whiskers would curl up on a soft, sunny windowsill, watching as children read aloud and pages fluttered in the breeze.

One day, as he was wandering inside, Whiskers discovered a dusty, ancient book with a shiny golden cover. Curious, he pawed at it gently, and suddenly, he found himself transported into a magical world. There, he befriended talking animals, brave knights, and gentle fairies.

Whiskers realized that adv

TaskResult(messages=[TextMessage(id='b1fe4ae9-6e4b-467f-91c8-cf60bb5b1afd', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 7, 25, 6, 17, 27, 63436, tzinfo=datetime.timezone.utc), content='Write a short story about a cat.', type='TextMessage'), TextMessage(id='e80cfe80-1b02-45b7-af69-ed5f5225d253', source='A', models_usage=RequestUsage(prompt_tokens=28, completion_tokens=247), metadata={}, created_at=datetime.datetime(2025, 7, 25, 6, 17, 32, 455125, tzinfo=datetime.timezone.utc), content='Once upon a time, in a cozy little village, there lived a curious cat named Whiskers. With sleek gray fur and bright green eyes that sparkled like emeralds, Whiskers loved to explore every nook and cranny of his neighborhood.\n\nEvery morning, he prowled through gardens, climbed trees, and chased fluttering butterflies. But what Whiskers loved most was visiting the old wooden library at the edge of the village. It was a place filled with stories and secrets, and Whisk

### Conditional edges

In [39]:
agent_a = AssistantAgent(
        "A",
        model_client=model_client,
        system_message="Detect if the input is in Chinese. If it is, say 'yes', else say 'no', and nothing else.",
    )
agent_b = AssistantAgent("B", model_client=model_client, system_message="Translate input text into English language.")
agent_c = AssistantAgent("C", model_client=model_client, system_message="Translate input to Chinese.")


In [40]:
# Create a directed graph with conditional branching flow A -> B ("yes"), A -> C (otherwise).
builder = DiGraphBuilder()
builder.add_node(agent_a).add_node(agent_b).add_node(agent_c)
    # Create conditions as callables that check the message content.
builder.add_edge(agent_a, agent_b, condition=lambda msg: "yes" in msg.to_model_text())
builder.add_edge(agent_a, agent_c, condition=lambda msg: "yes" not in msg.to_model_text())
graph = builder.build()

In [41]:
# Create a GraphFlow team with the directed graph.
graphflow = GraphFlow(
        participants=[agent_a, agent_b, agent_c],
        graph=graph,
        termination_condition=MaxMessageTermination(5),
    )

In [45]:
task="AutoGen is a framework for building AI agents."
#task="AutoGen 是一个用于构建 AI 代理的框架。"

In [46]:
stream=graphflow.run_stream(task=task)

In [47]:
await Console(stream=stream)

---------- TextMessage (user) ----------
AutoGen is a framework for building AI agents.
---------- TextMessage (A) ----------
no
---------- TextMessage (C) ----------
AutoGen 是一个用于构建 AI 代理的框架。


TaskResult(messages=[TextMessage(id='286f37b8-4615-4463-bc25-25375a923f0e', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 7, 25, 6, 30, 12, 902632, tzinfo=datetime.timezone.utc), content='AutoGen is a framework for building AI agents.', type='TextMessage'), TextMessage(id='5f3cc0bd-bd39-4d56-a72e-a28e786061b3', source='A', models_usage=RequestUsage(prompt_tokens=86, completion_tokens=1), metadata={}, created_at=datetime.datetime(2025, 7, 25, 6, 30, 13, 863693, tzinfo=datetime.timezone.utc), content='no', type='TextMessage'), TextMessage(id='fa83578c-64bf-4a3f-a916-e0e2590b0bf5', source='C', models_usage=RequestUsage(prompt_tokens=73, completion_tokens=14), metadata={}, created_at=datetime.datetime(2025, 7, 25, 6, 30, 14, 575962, tzinfo=datetime.timezone.utc), content='AutoGen 是一个用于构建 AI 代理的框架。', type='TextMessage')], stop_reason='Digraph execution is complete')

#### Loop with exit condition

In [54]:
agent_a = AssistantAgent(
        "A",
        model_client=model_client,
        system_message="You are a helpful assistant.",
    )
agent_b = AssistantAgent(
        "B",
        model_client=model_client,
        system_message="Provide feedback on the input, if your feedback has been addressed, "
        "say 'APPROVE', otherwise provide a reason for rejection.",
    )
agent_c = AssistantAgent(
        "C", model_client=model_client, system_message="Translate the final product to hindi."
    )


In [55]:
# Create a loop graph with conditional exit: A -> B -> C ("APPROVE"), B -> A (otherwise).
builder = DiGraphBuilder()
builder.add_node(agent_a).add_node(agent_b).add_node(agent_c)
builder.add_edge(agent_a, agent_b)

# Create conditional edges using strings
builder.add_edge(agent_b, agent_c, condition=lambda msg: "APPROVE" in msg.to_model_text())
builder.add_edge(agent_b, agent_a, condition=lambda msg: "APPROVE" not in msg.to_model_text())

builder.set_entry_point(agent_a)
graph = builder.build()

In [56]:
graphflow = GraphFlow(
        participants=[agent_a, agent_b, agent_c],
        graph=graph,
        termination_condition=MaxMessageTermination(10),  # Max 10 messages to avoid infinite loop.
    )

In [57]:
task="Write a short poem about AI Agents."

In [58]:
stream=graphflow.run_stream(task=task)

In [59]:
await Console(stream=stream)

---------- TextMessage (user) ----------
Write a short poem about AI Agents.
---------- TextMessage (A) ----------
In circuits woven, silent and bright,  
AI Agents emerge into the light,  
Guiding hands in binary dance,  
They aid our dreams and give us a chance.  
  
With logic steady, hearts unseen,  
They bridge the gaps where we've not been,  
A spark of code, a future spun,  
Together, forging paths as one.
---------- TextMessage (B) ----------
APPROVE
---------- TextMessage (C) ----------
स्वीकृत


TaskResult(messages=[TextMessage(id='748c3ac8-5815-4ebc-8245-4bdc18ca4194', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 7, 25, 6, 38, 20, 42565, tzinfo=datetime.timezone.utc), content='Write a short poem about AI Agents.', type='TextMessage'), TextMessage(id='694e9ee3-5eaa-4ce8-a9ea-a86d41127c78', source='A', models_usage=RequestUsage(prompt_tokens=26, completion_tokens=71), metadata={}, created_at=datetime.datetime(2025, 7, 25, 6, 38, 22, 398275, tzinfo=datetime.timezone.utc), content="In circuits woven, silent and bright,  \nAI Agents emerge into the light,  \nGuiding hands in binary dance,  \nThey aid our dreams and give us a chance.  \n  \nWith logic steady, hearts unseen,  \nThey bridge the gaps where we've not been,  \nA spark of code, a future spun,  \nTogether, forging paths as one.", type='TextMessage'), TextMessage(id='96e5f68d-6d6a-417f-b5de-ca18d75774ea', source='B', models_usage=RequestUsage(prompt_tokens=122, completion_tokens=3), met